In [1]:
#!/usr/bin/env python
import sys, os
import argparse

import numpy as np
import csv, yaml

import h5py
import torch
import torch.nn as nn

sys.path.append("./python")

from models.allModels import *
import easydict
import matplotlib.pyplot as plt

In [2]:
args = easydict.EasyDict({
    "config" : 'config_even_eval.yaml',
    "output" : 'test1592',
  
    "device" : 0,
    "epoch" : 1,
    "batch" : 10000000,
    "lr" : 0.0001,
    "seed" : 12345,
    "kernel_size" : 11,
    "model" : '1DCNN3FC2',
    "ratio" : 1.0,
    "runnum" : 1592,
    "rho" : 1.4,
    "vtz" : 1.0,
    "odd" : 1,
    "minvalue" : 400,
    "dvertex" : 1,
  
   

})

In [3]:

config = yaml.load(open(args.config).read(), Loader=yaml.FullLoader)
config['training']['learningRate'] = float(config['training']['learningRate'])
if args.seed: config['training']['randomSeed1'] = args.seed
if args.epoch: config['training']['epoch'] = args.epoch
if args.lr: config['training']['learningRate'] = args.lr

torch.set_num_threads(os.cpu_count())
if torch.cuda.is_available() and args.device >= 0: torch.cuda.set_device(args.device)
if not os.path.exists('result/' + args.output): os.makedirs('result/' + args.output)

##### Define dataset instance #####
from dataset.WFCh2Dataset_each_norm_involve_raw import *

##### Define dataset instance #####
dset = WFCh2Dataset_each_norm_involve_raw(channel=config['format']['channel'], output = 'result/' + args.output)
for sampleInfo in config['samples']:
    if 'ignore' in sampleInfo and sampleInfo['ignore']: continue
    name = sampleInfo['name']
    if args.odd == 1:
        in_path = '/store/hep/users/yewzzang/JSNS2/com_data/r00'+str(args.runnum)+'/'+name+'_cut_even_Rho_'+str(args.rho)+'_ZL_'+str(args.vtz)+'_noDIN/*.h5'
    elif args.odd == 2:
        in_path = '/store/hep/users/yewzzang/JSNS2/com_data/r00'+str(args.runnum)+'/'+name+'_cut_odd_Rho_'+str(args.rho)+'_ZL_'+str(args.vtz)+'_noDIN/*.h5'
    
    else: ## total training
        in_path = '/store/hep/users/yewzzang/JSNS2/com_data/r00'+str(args.runnum)+'/'+name+'_cut_Rho_'+str(args.rho)+'_ZL_'+str(args.vtz)+'_noDIN/*.h5'
    print(in_path)

    dset.addSample(name, in_path, weight=sampleInfo['xsec']/sampleInfo['ngen'])
    dset.setProcessLabel(name, sampleInfo['label'])
dset.initialize()

lengths = [int(x*len(dset)) for x in config['training']['splitFractions']]
lengths.append(len(dset)-sum(lengths))
torch.manual_seed(config['training']['randomSeed1'])
kwargs = {'num_workers':min(config['training']['nDataLoaders'], os.cpu_count()),
          'batch_size':args.batch, 'pin_memory':False}
from torch.utils.data import DataLoader

testLoader = DataLoader(dset, **kwargs)


torch.manual_seed(torch.initial_seed())

##### Define model instance #####
from models.allModels import *
#model = WF1DCNNModel(nChannel=dset.nCh, nPoint=dset.nPt)
model = torch.load('result/test1592/model.pth', map_location='cpu')
model.load_state_dict(torch.load('result/test1592/weight.pth', map_location='cpu'))

model.fc.add_module('output', torch.nn.Sigmoid())
device = 'cpu'



/store/hep/users/yewzzang/JSNS2/com_data/r001592/ME_cut_even_Rho_1.4_ZL_1.0_noDIN/*.h5
/store/hep/users/yewzzang/JSNS2/com_data/r001592/FN_cut_even_Rho_1.4_ZL_1.0_noDIN/*.h5
     procName                                           fileName  weight  \
0          ME  /store/hep/users/yewzzang/JSNS2/com_data/r0015...     1.0   
1          ME  /store/hep/users/yewzzang/JSNS2/com_data/r0015...     1.0   
2          ME  /store/hep/users/yewzzang/JSNS2/com_data/r0015...     1.0   
3          ME  /store/hep/users/yewzzang/JSNS2/com_data/r0015...     1.0   
4          ME  /store/hep/users/yewzzang/JSNS2/com_data/r0015...     1.0   
...       ...                                                ...     ...   
6004       FN  /store/hep/users/yewzzang/JSNS2/com_data/r0015...     1.0   
6005       FN  /store/hep/users/yewzzang/JSNS2/com_data/r0015...     1.0   
6006       FN  /store/hep/users/yewzzang/JSNS2/com_data/r0015...     1.0   
6007       FN  /store/hep/users/yewzzang/JSNS2/com_data/r0015...  

In [ ]:
from tqdm import tqdm
labels, preds = [], []
weights = []
procIdxs = []
fileIdxs = []
idxs = []
model.eval()
val_loss, val_acc = 0., 0.
# for i, (data, label0, weight, rescale, procIdx, fileIdx, idx, dT, dVertex, vertexX, vertexY, vertexZ) in enumerate(tqdm(testLoader)):

# for i, (data, label0, weight, rescale, procIdx, fileIdx, idx) in enumerate(tqdm(testLoader)):
for i, (data, label0, weight, rescale, procIdx, fileIdx, idx, dVertexx, minvaluee, pChargee,vertexx,vertexy,vertexz) in enumerate(tqdm(testLoader)):
    data = data


    label = label0
    label0 = label0.reshape(-1)[()].numpy()
    rescale = rescale.float()
    weight = weight.float()*rescale

    pred = model(data)

    #weight = np.ones(len(label0))
    #weight[label0==1] = dset.classWeight.item()

    labels.extend([x.item() for x in label])
    weights.extend([x.item() for x in weight])
    preds.extend([x.item() for x in pred.view(-1)])
    procIdxs.extend([x.item() for x in procIdx])
    fileIdxs.extend([x.item() for x in fileIdx])
    idxs.extend([x.item() for x in idx])
df = pd.DataFrame({'label':labels, 'prediction':preds, 'weight':weights, 'procIdx':procIdxs, 'fileIdx':fileIdxs, 'idx':idxs})

fPred = 'result/' + args.output + '/' + args.output + '.csv'
df.to_csv(fPred, index=False)


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:


##### Draw ROC curve #####
si_path = 'result/' + args.output + '/sampleInfo.csv'
si = pd.read_csv(si_path)
fPred = 'result/' + args.output + '/' + args.output + '.csv'
info = pd.read_csv(fPred)

info_numpy = np.array(info)
si_numpy = np.array(si)

preds = []
labels = []
dVertexs = []
dTs = []
vertexXs = []
vertexYs = []
vertexZs = []
for i in range(len(info_numpy)):

    label = info_numpy[i][0]
    pred = info_numpy[i][1]

    fileidx = info_numpy[i][4]
    
    filename = si_numpy[int(fileidx)][2]
    
    
    idx = info_numpy[i][5]

    
    data = h5py.File(filename,'r')

    dT = data['events']['dT'][idx]
    dVertex = data['events']['dVertex'][idx]
    vertexX = data['events']['vertexX'][idx]
    vertexY = data['events']['vertexY'][idx]
    vertexZ = data['events']['vertexZ'][idx]
    
    
    preds.append(pred)
    labels.append(label)
    dVertexs.append(dVertex)
    dTs.append(dT)
#     print(dTs.type)
    vertexXs.append(vertexX)
    vertexYs.append(vertexY)
    vertexZs.append(vertexZ)

preds = np.array(preds)
labels = np.array(labels)
dVertexs = np.array(dVertexs)
dTs = np.array(dTs)
vertexXs = np.array(vertexXs)
vertexYs = np.array(vertexYs)
vertexZs = np.array(vertexZs)


ME_label = []
ME_dVertex = []
ME_dT = []
ME_vertexX = []
ME_vertexY = []
ME_vertexZ = []
ME_pred = []


FN_label = []
FN_dVertex = []
FN_dT = []
FN_vertexX = []
FN_vertexY = []
FN_vertexZ = []
FN_pred = []

for i in range(len(preds)):
    if labels[i] == 1:
        ME_label.append(labels[i])
        ME_dVertex.append(dVertexs[i])
        ME_dT.append(dTs[i])
        ME_vertexX.append(vertexXs[i])
        ME_vertexY.append(vertexYs[i])
        ME_vertexZ.append(vertexZs[i])
        ME_pred.append(preds[i])
    else:
   
        FN_label.append(labels[i])
        FN_dVertex.append(dVertexs[i])
        FN_dT.append(dTs[i])
        FN_vertexX.append(vertexXs[i])
        FN_vertexY.append(vertexYs[i])
        FN_vertexZ.append(vertexZs[i])
        FN_pred.append(preds[i])
        
ME_sig = -np.log((1/np.array(ME_pred))-1)
FN_sig = -np.log((1/np.array(FN_pred))-1)
FN_over_0 = 100*np.sum(FN_sig > 0)/len(FN_sig)
ME_under_0 = 100*np.sum(ME_sig < 0)/len(ME_sig)


##################plot CNN score distribution figure
plt.hist(FN_sig, bins = 100, range = (-20, 20), density = True, color ='r',histtype = 'step')
plt.hist(ME_sig, bins = 100, range = (-20, 20), density = True, color ='b',histtype = 'step')
# plt.text(10, 0.15,'FN > 0' + str(FN_over_0), fontsize = 20)
# plt.text(10, 0.1, 'ME < 0' + str(ME_under_0), fontsize = 20)


FN_over = '%.2f' %FN_over_0
ME_under = '%.2f' %ME_under_0

label = ['FN : FN > 0 = '+FN_over+'%', 
         'ME : ME < 0 = '+ME_under+'%']


leg = plt.legend(label, loc = 'best', frameon=False)

leg_lines = leg.get_lines()
leg_texts = leg.get_texts()

plt.setp(leg_lines, linewidth=15)
plt.setp(leg_texts, fontsize=15)

plt.title('')
plt.show()



num_FN = len(FN_dT)
num_ME = len(ME_dT)
###########plot dT
plt.hist(np.array(FN_dT)*0.001, bins = 100, color= 'r', alpha = 0.5, density = True, histtype = 'step')
plt.hist(np.array(ME_dT)*0.001, bins = 100, color= 'b', alpha = 0.5, density = True, histtype = 'step')

plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
plt.xlabel("\u03BCs", fontsize=15, loc='right')
plt.show()


###########plot dVertex
plt.hist(np.array(FN_dVertex)*100, bins = 80, color= 'r', alpha = 0.5, density = True, histtype = 'step')
plt.hist(np.array(ME_dVertex)*100, bins = 80, color= 'b', alpha = 0.5, density = True, histtype = 'step')

plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
plt.xlabel("cm", fontsize=15, loc='right')
plt.show()



##############plot vertexZ
plt.hist(np.array(FN_vertexZ)*100, bins = 80, color= 'r', alpha = 0.5, density = True, histtype = 'step')
plt.hist(np.array(ME_vertexZ)*100, bins = 80, color= 'b', alpha = 0.5, density = True, histtype = 'step')

plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
plt.xlabel("cm", fontsize=15, loc='right')
plt.show()


##############plot R2
plt.hist((np.array(FN_vertexX)**2+np.array(FN_vertexY)**2)*100, bins = 80, color= 'r', alpha = 0.5, density = True, histtype = 'step')
plt.hist((np.array(ME_vertexX)**2+np.array(ME_vertexY)**2)*100, bins = 80, color= 'b', alpha = 0.5, density = True, histtype = 'step')

plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
plt.xlabel("cm", fontsize=15, loc='right')
plt.show()


####################################################################################

list_range = np.arange(0,1,0.001)


for i in range(len(list_range)):
    a = len(np.array(ME_pred)[np.array(ME_pred)>list_range[i]])/len(np.array(ME_pred))
#     print(list_range[i])
    
    if (a > 0.99):
     
        eff_99 = list_range[i]
        continue
        
    if (a > 0.95):
        
        eff_95 = list_range[i]
        continue
        
    if (a > 0.90):
       
        eff_90 = list_range[i]
        continue
##### eff 0.5 pred
eff_50_ME = len(np.array(ME_pred)[np.array(ME_pred)>0.5])/len(np.array(ME_pred))
eff_50_FN = len(np.array(FN_pred)[np.array(FN_pred)>0.5])/len(np.array(FN_pred))
num_50_ME = num_ME*len(np.array(ME_pred)[np.array(ME_pred)>0.5])/len(np.array(ME_pred))
num_50_FN = num_FN*len(np.array(FN_pred)[np.array(FN_pred)>0.5])/len(np.array(FN_pred))



##### eff 99% efficient
eff_99_ME = len(np.array(ME_pred)[np.array(ME_pred)>eff_99])/len(np.array(ME_pred))
eff_99_FN = len(np.array(FN_pred)[np.array(FN_pred)>eff_99])/len(np.array(FN_pred))
num_99_ME = num_ME*len(np.array(ME_pred)[np.array(ME_pred)>eff_99])/len(np.array(ME_pred))
num_99_FN = num_FN*len(np.array(FN_pred)[np.array(FN_pred)>eff_99])/len(np.array(FN_pred))




##### eff 95% efficient
eff_95_ME = len(np.array(ME_pred)[np.array(ME_pred)>eff_95])/len(np.array(ME_pred))
eff_95_FN = len(np.array(FN_pred)[np.array(FN_pred)>eff_95])/len(np.array(FN_pred))
num_95_ME = num_ME*len(np.array(ME_pred)[np.array(ME_pred)>eff_95])/len(np.array(ME_pred))
num_95_FN = num_FN*len(np.array(FN_pred)[np.array(FN_pred)>eff_95])/len(np.array(FN_pred))


##### eff 90% efficient
eff_90_ME = len(np.array(ME_pred)[np.array(ME_pred)>eff_90])/len(np.array(ME_pred))
eff_90_FN = len(np.array(FN_pred)[np.array(FN_pred)>eff_90])/len(np.array(FN_pred))
num_90_ME = num_ME*len(np.array(ME_pred)[np.array(ME_pred)>eff_90])/len(np.array(ME_pred))
num_90_FN = num_FN*len(np.array(FN_pred)[np.array(FN_pred)>eff_90])/len(np.array(FN_pred))


 
# # f = open('result/' + args.output + '/' + args.output + '_efficiency.txt','w')
# print('         |   90%   |   95%   |   99%   |   mid   |',file=f)
# print('--------------------------------------------------',file=f)
# print('  ME_eff |','%.4f  |'%eff_90_ME,'%.4f  |'%eff_95_ME,'%.4f  |'%eff_99_ME,'%.4f  |'%eff_50_ME,file=f)
# print('--------------------------------------------------',file=f)
# print('  FN_eff |','%.4f  |'%eff_90_FN,'%.4f  |'%eff_95_FN,'%.4f  |'%eff_99_FN,'%.4f  |'%eff_50_FN,file=f)
# print('--------------------------------------------------',file=f)
# print('ME_remain|','%7d'%int(num_90_ME),'|','%7d'%int(num_95_ME),'|','%7d'%int(num_99_ME),'|','%7d'%int(num_50_ME),'|',file=f)
# print('--------------------------------------------------',file=f)
# print('FN_remain|','%7d'%int(num_90_FN),'|','%7d'%int(num_95_FN),'|','%7d'%int(num_99_FN),'|','%7d'%int(num_50_FN),'|',file=f)
# print('--------------------------------------------------',file=f)
# print('CNN score|','','%.3f'%eff_90,' | ','%.3f'%eff_95,' | ','%.3f'%eff_99,' |  ',0.5,'  |',file=f)
# print('==================================================',file=f)
# print('         |   FN    |   ME    |',file=f)
# print('# data   |','%7d'%len(FN_dT),'|', '%7d'%len(ME_dT),'|',file=f)

# f.close()



###############plot evaluation 

##### Draw ROC curve #####
from sklearn.metrics import roc_curve, roc_auc_score
from matplotlib.cbook import get_sample_data

df = pd.read_csv(fPred)
tpr, fpr, thr = roc_curve(df['label'], df['prediction'], sample_weight=df['weight'], pos_label=0)
auc = roc_auc_score(df['label'], df['prediction'], sample_weight=df['weight'])


df_bkg = df[df.label==0]
df_sig = df[df.label==1]
plt.rcParams['figure.figsize'] = (10, 10)

plt.hist(df_bkg['prediction']*100, weights=df_bkg['weight'], histtype='step', 
         density=True, bins=50, color='red', linewidth=3)

plt.hist(df_sig['prediction']*100, weights=df_sig['weight'], histtype='step', 
         density=True, bins=50, color='blue', linewidth=3)



plt.xticks(np.arange(0, 101, step=20),["{}".format(x*0.01) for x in np.arange(0, 101,step=20)],fontsize = 20)
plt.yticks(fontsize = 20)
plt.xlabel("CNN score", fontsize=35, loc='right')
plt.ylabel("Normalized", fontsize=35, loc='top')
plt.xlim(0, 100)
label = ['Fast Neutron', 'Michel Electrons']

leg = plt.legend(label, loc = 'upper center', frameon=False)

leg_lines = leg.get_lines()
leg_texts = leg.get_texts()

plt.setp(leg_lines, linewidth=30)
plt.setp(leg_texts, fontsize=30)

plt.show()




###################### plot AUC


plt.rcParams['figure.figsize'] = (10, 10)



plt.xticks(fontsize = 20)
plt.yticks(fontsize = 20)



plt.plot(fpr*100, tpr*100, label='AUC = %.3f' % (auc))
plt.plot(eff_99_FN*100, eff_99_ME*100,'*r', markersize=40)

plt.plot(eff_95_FN*100, eff_95_ME*100,'*g', markersize=40)

plt.plot(eff_90_FN*100, eff_90_ME*100,'*b', markersize=40)

plt.plot(eff_50_FN*100, eff_50_ME*100,'*k', markersize=40)

plt.xlabel('FN efficiency (%)', fontsize=35, loc='right')
plt.ylabel('ME efficiency (%)', fontsize=35, loc='top')


plt.xlim(0, 100)
plt.ylim(0, 100)

plt.grid()
print_auc = '%.3f' %auc
print_eff_99 = '%.3f' %eff_99
print_eff_95 = '%.3f' %eff_95
print_eff_90 = '%.3f' %eff_90
label = ['AUC = '+print_auc,'99% WP(score='+print_eff_99+')', '95% WP(score='+print_eff_95+')', '90% WP(score='+print_eff_90+')','Middle WP (score=0.5)']

leg = plt.legend(label, loc = 'right', frameon=False)

 
leg_lines = leg.get_lines()
leg_texts = leg.get_texts()

plt.setp(leg_lines, linewidth=15)
plt.setp(leg_texts, fontsize=30)

plt.show()


In [ ]:

for epoch in range(nEpoch):


    for i, (data, label0, weight, rescale, procIdx, fileIdx, idx, dVertexx, minvaluee, pChargee,vertexx,vertexy,vertexz) in enumerate(tqdm(testLoader, desc='epoch %d/%d' % (epoch+1, nEpoch))):
        data = data.to(device)
        label = label0.float().to(device)
        weight = (weight*rescale).float().to(device)


In [ ]:
data = data.to("cpu")
label = label.to("cpu")
data_avg =np.average(data,axis=1)

In [ ]:
plt.plot(np.average(data_avg[label==0],axis=0).T,color='red',label='FN')
plt.plot(np.average(data_avg[label==1],axis=0).T,color='blue',label='ME')
plt.plot(np.average(data_avg[label==1],axis=0).T-np.average(data_avg[label==0],axis=0).T,color='green',label='dif')
plt.plot(np.zeros(208),color='black')
plt.legend()
plt.show()

In [ ]:
args = easydict.EasyDict({
    "config" : 'config_even_eval.yaml',
    "output" : 'test_220330',
  
    "device" : 0,
    "epoch" : 1,
    "batch" : 10000000,
    "lr" : 0.0001,
    "seed" : 12345,
    "kernel_size" : 11,
    "model" : '1DCNN3FC2',
    "ratio" : 1.0,
    "runnum" : 1563,
    "rho" : 1.6,
    "vtz" : 1.0,
    "odd" : 1,
    "minvalue" : 400,
    "dvertex" : 1,
  
   

})

config = yaml.load(open(args.config).read(), Loader=yaml.FullLoader)
config['training']['learningRate'] = float(config['training']['learningRate'])
if args.seed: config['training']['randomSeed1'] = args.seed
if args.epoch: config['training']['epoch'] = args.epoch
if args.lr: config['training']['learningRate'] = args.lr

torch.set_num_threads(os.cpu_count())
if torch.cuda.is_available() and args.device >= 0: torch.cuda.set_device(args.device)
if not os.path.exists('result/' + args.output): os.makedirs('result/' + args.output)

##### Define dataset instance #####
from dataset.WFCh2Dataset_each_norm_involve_raw import *

##### Define dataset instance #####
dset = WFCh2Dataset_each_norm_involve_raw(channel=config['format']['channel'], output = 'result/' + args.output)
for sampleInfo in config['samples']:
    if 'ignore' in sampleInfo and sampleInfo['ignore']: continue
    name = sampleInfo['name']
    if args.odd == 1:
        in_path = '/store/hep/users/yewzzang/JSNS2/com_data/r00'+str(args.runnum)+'/'+name+'_cut_even_Rho_'+str(args.rho)+'_ZL_'+str(args.vtz)+'/*.h5'
    elif args.odd == 2:
        in_path = '/store/hep/users/yewzzang/JSNS2/com_data/r00'+str(args.runnum)+'/'+name+'_cut_odd_Rho_'+str(args.rho)+'_ZL_'+str(args.vtz)+'/*.h5'
    
    else: ## total training
        in_path = '/store/hep/users/yewzzang/JSNS2/com_data/r00'+str(args.runnum)+'/'+name+'_cut_Rho_'+str(args.rho)+'_ZL_'+str(args.vtz)+'/*.h5'
    print(in_path)

    dset.addSample(name, in_path, weight=sampleInfo['xsec']/sampleInfo['ngen'])
    dset.setProcessLabel(name, sampleInfo['label'])
dset.initialize()

lengths = [int(x*len(dset)) for x in config['training']['splitFractions']]
lengths.append(len(dset)-sum(lengths))
torch.manual_seed(config['training']['randomSeed1'])
kwargs = {'num_workers':min(config['training']['nDataLoaders'], os.cpu_count()),
          'batch_size':args.batch, 'pin_memory':False}
from torch.utils.data import DataLoader

testLoader = DataLoader(dset, **kwargs)


torch.manual_seed(torch.initial_seed())

device = 'cpu'


from sklearn.metrics import accuracy_score
from tqdm import tqdm
bestState, bestLoss = {}, 1e9
train = {'loss':[], 'acc':[], 'val_loss':[], 'val_acc':[]}
nEpoch = config['training']['epoch']



for epoch in range(nEpoch):


    for i, (data, label0, weight, rescale, procIdx, fileIdx, idx, dVertexx, minvaluee, pChargee,vertexx,vertexy,vertexz) in enumerate(tqdm(testLoader, desc='epoch %d/%d' % (epoch+1, nEpoch))):
        data_16 = data.to(device)
        label_16 = label0.float().to(device)
        weight_16 = (weight*rescale).float().to(device)


In [ ]:
data_16 = data_16.to("cpu")
label_16 = label_16.to("cpu")
data_avg_16 =np.average(data_16,axis=1)

In [ ]:
plt.plot(np.average(data_avg_16[label_16==0],axis=0).T,color='red',label='FN')
plt.plot(np.average(data_avg_16[label_16==1],axis=0).T,color='blue',label='ME')
plt.plot(np.average(data_avg_16[label_16==1],axis=0).T-np.average(data_avg_16[label_16==0],axis=0).T,color='green',label='dif')
plt.plot(np.zeros(208),color='black')
plt.legend()
plt.show()

In [ ]:
args = easydict.EasyDict({
    "config" : 'config_even_eval.yaml',
    "output" : 'test_220330',
  
    "device" : 0,
    "epoch" : 1,
    "batch" : 10000000,
    "lr" : 0.0001,
    "seed" : 12345,
    "kernel_size" : 11,
    "model" : '1DCNN3FC2',
    "ratio" : 1.0,
    "runnum" : 1563,
    "rho" : 1.8,
    "vtz" : 1.0,
    "odd" : 1,
    "minvalue" : 400,
    "dvertex" : 1,
  
   

})

config = yaml.load(open(args.config).read(), Loader=yaml.FullLoader)
config['training']['learningRate'] = float(config['training']['learningRate'])
if args.seed: config['training']['randomSeed1'] = args.seed
if args.epoch: config['training']['epoch'] = args.epoch
if args.lr: config['training']['learningRate'] = args.lr

torch.set_num_threads(os.cpu_count())
if torch.cuda.is_available() and args.device >= 0: torch.cuda.set_device(args.device)
if not os.path.exists('result/' + args.output): os.makedirs('result/' + args.output)

##### Define dataset instance #####
from dataset.WFCh2Dataset_each_norm_involve_raw import *

##### Define dataset instance #####
dset = WFCh2Dataset_each_norm_involve_raw(channel=config['format']['channel'], output = 'result/' + args.output)
for sampleInfo in config['samples']:
    if 'ignore' in sampleInfo and sampleInfo['ignore']: continue
    name = sampleInfo['name']
    if args.odd == 1:
        in_path = '/store/hep/users/yewzzang/JSNS2/com_data/r00'+str(args.runnum)+'/'+name+'_cut_even_Rho_'+str(args.rho)+'_ZL_'+str(args.vtz)+'/*.h5'
    elif args.odd == 2:
        in_path = '/store/hep/users/yewzzang/JSNS2/com_data/r00'+str(args.runnum)+'/'+name+'_cut_odd_Rho_'+str(args.rho)+'_ZL_'+str(args.vtz)+'/*.h5'
    
    else: ## total training
        in_path = '/store/hep/users/yewzzang/JSNS2/com_data/r00'+str(args.runnum)+'/'+name+'_cut_Rho_'+str(args.rho)+'_ZL_'+str(args.vtz)+'/*.h5'
    print(in_path)

    dset.addSample(name, in_path, weight=sampleInfo['xsec']/sampleInfo['ngen'])
    dset.setProcessLabel(name, sampleInfo['label'])
dset.initialize()

lengths = [int(x*len(dset)) for x in config['training']['splitFractions']]
lengths.append(len(dset)-sum(lengths))
torch.manual_seed(config['training']['randomSeed1'])
kwargs = {'num_workers':min(config['training']['nDataLoaders'], os.cpu_count()),
          'batch_size':args.batch, 'pin_memory':False}
from torch.utils.data import DataLoader

testLoader = DataLoader(dset, **kwargs)


torch.manual_seed(torch.initial_seed())

device = 'cpu'


from sklearn.metrics import accuracy_score
from tqdm import tqdm
bestState, bestLoss = {}, 1e9
train = {'loss':[], 'acc':[], 'val_loss':[], 'val_acc':[]}
nEpoch = config['training']['epoch']



for epoch in range(nEpoch):


    for i, (data, label0, weight, rescale, procIdx, fileIdx, idx, dVertexx, minvaluee, pChargee,vertexx,vertexy,vertexz) in enumerate(tqdm(testLoader, desc='epoch %d/%d' % (epoch+1, nEpoch))):
        data_18 = data.to(device)
        label_18 = label0.float().to(device)
        weight_18 = (weight*rescale).float().to(device)


In [ ]:
data_18 = data_18.to("cpu")
label_18 = label_18.to("cpu")
data_avg_18 =np.average(data_18,axis=1)

In [ ]:
plt.plot(np.average(data_avg_18[label_18==0],axis=0).T,color='red',label='FN')
plt.plot(np.average(data_avg_18[label_18==1],axis=0).T,color='blue',label='ME')
plt.plot(np.average(data_avg_18[label_18==1],axis=0).T-np.average(data_avg_18[label_18==0],axis=0).T,color='green',label='dif')
plt.plot(np.zeros(208),color='black')
plt.legend()
plt.show()

In [ ]:
plt.plot(np.average(data_avg[label==1],axis=0).T,color='red',label='ME_DIN')

plt.plot(np.average(data_avg_16[label_16==1],axis=0).T,color='blue',label='ME_nonDIN candidate')

plt.plot(np.average(data_avg_18[label_18==1],axis=0).T,color='green',label='ME_noDIN')
plt.plot(np.zeros(208),color='black')
plt.legend()
plt.show()



In [ ]:
plt.plot(np.average(data_avg[label==0],axis=0).T,color='red',label='FN_DIN')

plt.plot(np.average(data_avg_16[label_16==0],axis=0).T,color='blue',label='FN_nonDIN candidate')

plt.plot(np.average(data_avg_18[label_18==0],axis=0).T,color='green',label='FN_noDIN')
plt.plot(np.zeros(208),color='black')
plt.legend()
plt.show()



In [ ]:
plt.plot(np.average(data_avg[label==1],axis=0).T/max(np.average(data_avg[label==1],axis=0)),color='red',label='ME_DIN')

plt.plot(np.average(data_avg_16[label_16==1],axis=0).T/max(np.average(data_avg_16[label_16==1],axis=0)),color='blue',label='ME_nonDIN candidate')

plt.plot(np.average(data_avg_18[label_18==1],axis=0).T/max(np.average(data_avg_18[label_18==1],axis=0)),color='green',label='ME_noDIN')
plt.plot(np.zeros(208),color='black')
plt.legend()
plt.show()



In [ ]:
plt.plot(np.average(data_avg[label==0],axis=0).T/max(np.average(data_avg[label==0],axis=0)),color='red',label='FN_DIN')

plt.plot(np.average(data_avg_16[label_16==0],axis=0).T/max(np.average(data_avg_16[label_16==0],axis=0)),color='blue',label='FN_nonDIN candidate')

plt.plot(np.average(data_avg_18[label_18==0],axis=0).T/max(np.average(data_avg_18[label_18==0],axis=0)),color='green',label='FN_noDIN')
plt.plot(np.zeros(208),color='black')
plt.legend()
plt.show()



In [ ]:
args = easydict.EasyDict({
    "config" : 'config_total.yaml',
    "output" : 'test_220330',
  
    "device" : 0,
    "epoch" : 1,
    "batch" : 10000000,
    "lr" : 0.0001,
    "seed" : 12345,
    "kernel_size" : 11,
    "model" : '1DCNN3FC2',
    "ratio" : 1.0,
    "runnum" : 1500,
    "rho" : 1.4,
    "vtz" : 1.0,
    "odd" : 1,
    "minvalue" : 400,
    "dvertex" : 1,
  
   

})

config = yaml.load(open(args.config).read(), Loader=yaml.FullLoader)
config['training']['learningRate'] = float(config['training']['learningRate'])
if args.seed: config['training']['randomSeed1'] = args.seed
if args.epoch: config['training']['epoch'] = args.epoch
if args.lr: config['training']['learningRate'] = args.lr

torch.set_num_threads(os.cpu_count())
if torch.cuda.is_available() and args.device >= 0: torch.cuda.set_device(args.device)
if not os.path.exists('result/' + args.output): os.makedirs('result/' + args.output)

##### Define dataset instance #####
from dataset.WFCh2Dataset_each_norm_involve_raw import *

##### Define dataset instance #####
dset = WFCh2Dataset_each_norm_involve_raw(channel=config['format']['channel'], output = 'result/' + args.output)
for sampleInfo in config['samples']:
    if 'ignore' in sampleInfo and sampleInfo['ignore']: continue
    name = sampleInfo['name']
    if args.odd == 1:
        in_path = '/store/hep/users/yewzzang/JSNS2/com_data/r00'+str(args.runnum)+'/'+name+'_cut_even_Rho_'+str(args.rho)+'_ZL_'+str(args.vtz)+'/*.h5'
    elif args.odd == 2:
        in_path = '/store/hep/users/yewzzang/JSNS2/com_data/r00'+str(args.runnum)+'/'+name+'_cut_odd_Rho_'+str(args.rho)+'_ZL_'+str(args.vtz)+'/*.h5'
    
    else: ## total training
        in_path = '/store/hep/users/yewzzang/JSNS2/com_data/r00'+str(args.runnum)+'/'+name+'_cut_Rho_'+str(args.rho)+'_ZL_'+str(args.vtz)+'/*.h5'
    print(in_path)

    dset.addSample(name, in_path, weight=sampleInfo['xsec']/sampleInfo['ngen'])
    dset.setProcessLabel(name, sampleInfo['label'])
dset.initialize()

lengths = [int(x*len(dset)) for x in config['training']['splitFractions']]
lengths.append(len(dset)-sum(lengths))
torch.manual_seed(config['training']['randomSeed1'])
kwargs = {'num_workers':min(config['training']['nDataLoaders'], os.cpu_count()),
          'batch_size':args.batch, 'pin_memory':False}
from torch.utils.data import DataLoader

testLoader = DataLoader(dset, **kwargs)


torch.manual_seed(torch.initial_seed())

device = 'cpu'


from sklearn.metrics import accuracy_score
from tqdm import tqdm
bestState, bestLoss = {}, 1e9
train = {'loss':[], 'acc':[], 'val_loss':[], 'val_acc':[]}
nEpoch = config['training']['epoch']



for epoch in range(nEpoch):


    for i, (data, label0, weight, rescale, procIdx, fileIdx, idx, dVertexx, minvaluee, pChargee,vertexx,vertexy,vertexz) in enumerate(tqdm(testLoader, desc='epoch %d/%d' % (epoch+1, nEpoch))):
        data_1500 = data.to(device)
        label_1500 = label0.float().to(device)
        weight_1500 = (weight*rescale).float().to(device)


In [ ]:
data_1500 = data_1500.to("cpu")
label_1500 = label_1500.to("cpu")
data_avg_1500 =np.average(data_1500,axis=1)

In [ ]:
plt.plot(np.average(data_avg_1500[label_1500==0],axis=0).T,color='red',label='FN')
plt.plot(np.average(data_avg_1500[label_1500==1],axis=0).T,color='blue',label='ME')
plt.plot(np.average(data_avg_1500[label_1500==1],axis=0).T-np.average(data_avg_1500[label_1500==0],axis=0).T,color='green',label='dif')
plt.plot(np.zeros(208),color='black')
plt.legend()
plt.show()

In [ ]:
args = easydict.EasyDict({
    "config" : 'config_total.yaml',
    "output" : 'test_220330',
  
    "device" : 0,
    "epoch" : 1,
    "batch" : 10000000,
    "lr" : 0.0001,
    "seed" : 12345,
    "kernel_size" : 11,
    "model" : '1DCNN3FC2',
    "ratio" : 1.0,
    "runnum" : 1500,
    "rho" : 1.6,
    "vtz" : 1.0,
    "odd" : 1,
    "minvalue" : 400,
    "dvertex" : 1,
  
   

})

config = yaml.load(open(args.config).read(), Loader=yaml.FullLoader)
config['training']['learningRate'] = float(config['training']['learningRate'])
if args.seed: config['training']['randomSeed1'] = args.seed
if args.epoch: config['training']['epoch'] = args.epoch
if args.lr: config['training']['learningRate'] = args.lr

torch.set_num_threads(os.cpu_count())
if torch.cuda.is_available() and args.device >= 0: torch.cuda.set_device(args.device)
if not os.path.exists('result/' + args.output): os.makedirs('result/' + args.output)

##### Define dataset instance #####
from dataset.WFCh2Dataset_each_norm_involve_raw import *

##### Define dataset instance #####
dset = WFCh2Dataset_each_norm_involve_raw(channel=config['format']['channel'], output = 'result/' + args.output)
for sampleInfo in config['samples']:
    if 'ignore' in sampleInfo and sampleInfo['ignore']: continue
    name = sampleInfo['name']
    if args.odd == 1:
        in_path = '/store/hep/users/yewzzang/JSNS2/com_data/r00'+str(args.runnum)+'/'+name+'_cut_even_Rho_'+str(args.rho)+'_ZL_'+str(args.vtz)+'/*.h5'
    elif args.odd == 2:
        in_path = '/store/hep/users/yewzzang/JSNS2/com_data/r00'+str(args.runnum)+'/'+name+'_cut_odd_Rho_'+str(args.rho)+'_ZL_'+str(args.vtz)+'/*.h5'
    
    else: ## total training
        in_path = '/store/hep/users/yewzzang/JSNS2/com_data/r00'+str(args.runnum)+'/'+name+'_cut_Rho_'+str(args.rho)+'_ZL_'+str(args.vtz)+'/*.h5'
    print(in_path)

    dset.addSample(name, in_path, weight=sampleInfo['xsec']/sampleInfo['ngen'])
    dset.setProcessLabel(name, sampleInfo['label'])
dset.initialize()

lengths = [int(x*len(dset)) for x in config['training']['splitFractions']]
lengths.append(len(dset)-sum(lengths))
torch.manual_seed(config['training']['randomSeed1'])
kwargs = {'num_workers':min(config['training']['nDataLoaders'], os.cpu_count()),
          'batch_size':args.batch, 'pin_memory':False}
from torch.utils.data import DataLoader

testLoader = DataLoader(dset, **kwargs)


torch.manual_seed(torch.initial_seed())

device = 'cpu'


from sklearn.metrics import accuracy_score
from tqdm import tqdm
bestState, bestLoss = {}, 1e9
train = {'loss':[], 'acc':[], 'val_loss':[], 'val_acc':[]}
nEpoch = config['training']['epoch']



for epoch in range(nEpoch):


    for i, (data, label0, weight, rescale, procIdx, fileIdx, idx, dVertexx, minvaluee, pChargee,vertexx,vertexy,vertexz) in enumerate(tqdm(testLoader, desc='epoch %d/%d' % (epoch+1, nEpoch))):
        data_1500_16 = data.to(device)
        label_1500_16 = label0.float().to(device)
        weight_1500_16 = (weight*rescale).float().to(device)


In [ ]:
data_1500_16 = data_1500_16.to("cpu")
label_1500_16 = label_1500_16.to("cpu")
data_avg_1500_16 =np.average(data_1500_16,axis=1)

plt.plot(np.average(data_avg_1500_16[label_1500_16==0],axis=0).T,color='red',label='FN')
plt.plot(np.average(data_avg_1500_16[label_1500_16==1],axis=0).T,color='blue',label='ME')
plt.plot(np.average(data_avg_1500_16[label_1500_16==1],axis=0).T-np.average(data_avg_1500_16[label_1500_16==0],axis=0).T,color='green',label='dif')
plt.plot(np.zeros(208),color='black')
plt.legend()
plt.show()

In [ ]:
args = easydict.EasyDict({
    "config" : 'config_total.yaml',
    "output" : 'test_220330',
  
    "device" : 0,
    "epoch" : 1,
    "batch" : 10000000,
    "lr" : 0.0001,
    "seed" : 12345,
    "kernel_size" : 11,
    "model" : '1DCNN3FC2',
    "ratio" : 1.0,
    "runnum" : 1500,
    "rho" : 1.8,
    "vtz" : 1.0,
    "odd" : 1,
    "minvalue" : 400,
    "dvertex" : 1,
  
   

})

config = yaml.load(open(args.config).read(), Loader=yaml.FullLoader)
config['training']['learningRate'] = float(config['training']['learningRate'])
if args.seed: config['training']['randomSeed1'] = args.seed
if args.epoch: config['training']['epoch'] = args.epoch
if args.lr: config['training']['learningRate'] = args.lr

torch.set_num_threads(os.cpu_count())
if torch.cuda.is_available() and args.device >= 0: torch.cuda.set_device(args.device)
if not os.path.exists('result/' + args.output): os.makedirs('result/' + args.output)

##### Define dataset instance #####
from dataset.WFCh2Dataset_each_norm_involve_raw import *

##### Define dataset instance #####
dset = WFCh2Dataset_each_norm_involve_raw(channel=config['format']['channel'], output = 'result/' + args.output)
for sampleInfo in config['samples']:
    if 'ignore' in sampleInfo and sampleInfo['ignore']: continue
    name = sampleInfo['name']
    if args.odd == 1:
        in_path = '/store/hep/users/yewzzang/JSNS2/com_data/r00'+str(args.runnum)+'/'+name+'_cut_even_Rho_'+str(args.rho)+'_ZL_'+str(args.vtz)+'/*.h5'
    elif args.odd == 2:
        in_path = '/store/hep/users/yewzzang/JSNS2/com_data/r00'+str(args.runnum)+'/'+name+'_cut_odd_Rho_'+str(args.rho)+'_ZL_'+str(args.vtz)+'/*.h5'
    
    else: ## total training
        in_path = '/store/hep/users/yewzzang/JSNS2/com_data/r00'+str(args.runnum)+'/'+name+'_cut_Rho_'+str(args.rho)+'_ZL_'+str(args.vtz)+'/*.h5'
    print(in_path)

    dset.addSample(name, in_path, weight=sampleInfo['xsec']/sampleInfo['ngen'])
    dset.setProcessLabel(name, sampleInfo['label'])
dset.initialize()

lengths = [int(x*len(dset)) for x in config['training']['splitFractions']]
lengths.append(len(dset)-sum(lengths))
torch.manual_seed(config['training']['randomSeed1'])
kwargs = {'num_workers':min(config['training']['nDataLoaders'], os.cpu_count()),
          'batch_size':args.batch, 'pin_memory':False}
from torch.utils.data import DataLoader

testLoader = DataLoader(dset, **kwargs)


torch.manual_seed(torch.initial_seed())

device = 'cpu'


from sklearn.metrics import accuracy_score
from tqdm import tqdm
bestState, bestLoss = {}, 1e9
train = {'loss':[], 'acc':[], 'val_loss':[], 'val_acc':[]}
nEpoch = config['training']['epoch']



for epoch in range(nEpoch):


    for i, (data, label0, weight, rescale, procIdx, fileIdx, idx, dVertexx, minvaluee, pChargee,vertexx,vertexy,vertexz) in enumerate(tqdm(testLoader, desc='epoch %d/%d' % (epoch+1, nEpoch))):
        data_1500_18 = data.to(device)
        label_1500_18 = label0.float().to(device)
        weight_1500_18 = (weight*rescale).float().to(device)


In [ ]:
data_1500_18 = data_1500_18.to("cpu")
label_1500_18 = label_1500_18.to("cpu")
data_avg_1500_18 =np.average(data_1500_18,axis=1)

plt.plot(np.average(data_avg_1500_18[label_1500_18==0],axis=0).T,color='red',label='FN')
plt.plot(np.average(data_avg_1500_18[label_1500_18==1],axis=0).T,color='blue',label='ME')
plt.plot(np.average(data_avg_1500_18[label_1500_18==1],axis=0).T-np.average(data_avg_1500_18[label_1500_18==0],axis=0).T,color='green',label='dif')
plt.plot(np.zeros(208),color='black')
plt.legend()
plt.show()

In [ ]:
plt.plot(np.average(data_avg_1500[label_1500==1],axis=0).T,color='red',label='ME_1.4')

plt.plot(np.average(data_avg_1500_16[label_1500_16==1],axis=0).T,color='blue',label='ME_1.4-1.6')

plt.plot(np.average(data_avg_1500_18[label_1500_18==1],axis=0).T,color='green',label='ME_1.6-1.8')
plt.plot(np.zeros(208),color='black')
plt.legend()
plt.show()



In [ ]:
plt.plot(np.average(data_avg_1500[label_1500==0],axis=0).T,color='red',label='FN_1.4')

plt.plot(np.average(data_avg_1500_16[label_1500_16==0],axis=0).T,color='blue',label='FN_1.4-1.6')

plt.plot(np.average(data_avg_1500_18[label_1500_18==0],axis=0).T,color='green',label='FN_1.6-1.8')
plt.plot(np.zeros(208),color='black')
plt.legend()
plt.show()



In [ ]:
plt.plot(np.average(data_avg_1500[label_1500==1],axis=0).T/max(np.average(data_avg_1500[label_1500==1],axis=0)),color='red',label='ME_1.4')

plt.plot(np.average(data_avg_1500_16[label_1500_16==1],axis=0).T/max(np.average(data_avg_1500_16[label_1500_16==1],axis=0)),color='blue',label='ME_1.4-1.6')

plt.plot(np.average(data_avg_1500_18[label_1500_18==1],axis=0).T/max(np.average(data_avg_1500_18[label_1500_18==1],axis=0)),color='green',label='ME_1.6-1.8')
plt.plot(np.zeros(208),color='black')
plt.legend()
plt.show()



In [ ]:
plt.plot(np.average(data_avg_1500[label_1500==0],axis=0).T/max(np.average(data_avg_1500[label_1500==0],axis=0)),color='red',label='FN_1.4')

plt.plot(np.average(data_avg_1500_16[label_1500_16==0],axis=0).T/max(np.average(data_avg_1500_16[label_1500_16==0],axis=0)),color='blue',label='FN_1.4-1.6')

plt.plot(np.average(data_avg_1500_18[label_1500_18==0],axis=0).T/max(np.average(data_avg_1500_18[label_1500_18==0],axis=0)),color='green',label='FN_1.6-1.8')
plt.plot(np.zeros(208),color='black')
plt.legend()
plt.show()



In [ ]:
plt.plot(np.average(data_avg_1500[label_1500==1],axis=0).T-np.average(data_avg[label==1],axis=0).T,color='red',label='noDIN-DIN ME_1.4')

plt.plot(np.average(data_avg_1500_16[label_1500_16==1],axis=0).T-np.average(data_avg_16[label_16==1],axis=0).T,color='blue',label='noDIN-DIN ME_1.4-1.6')

plt.plot(np.average(data_avg_1500_18[label_1500_18==1],axis=0).T-np.average(data_avg_18[label_18==1],axis=0).T,color='green',label='noDIN-DIN ME_1.6-1.8')
plt.plot(np.zeros(208),color='black')
plt.legend()
plt.show()



In [ ]:
plt.plot(np.average(data_avg_1500[label_1500==0],axis=0).T-np.average(data_avg[label==0],axis=0).T,color='red',label='noDIN-DIN FN_1.4')

plt.plot(np.average(data_avg_1500_16[label_1500_16==0],axis=0).T-np.average(data_avg_16[label_16==0],axis=0).T,color='blue',label='noDIN-DIN FN_1.4-1.6')

plt.plot(np.average(data_avg_1500_18[label_1500_18==0],axis=0).T-np.average(data_avg_18[label_18==0],axis=0).T,color='green',label='noDIN-DIN FN_1.6-1.8')
plt.plot(np.zeros(208),color='black')
plt.legend()
plt.show()

